In [ ]:
# PASSO A PASSO
# Criar um grupo de usuario com permissao de Administrator
# Criar um usuario e atribuir ele ao grupo de Administrator
# Gerar uma Secret para o usuario
# Criar as credenciais no Google Colab para aws_access_key_id e aws_secret_access_key
# Criar uma role concedendo permissao do KDF (Kinesis Data Firehose) com o servico do S3 - Apenas para a demonstracao do Firehose

In [ ]:
!pip install boto3 awscli

In [5]:
import getpass
from google.colab import userdata

In [ ]:
# Entre com as informacoes da credencial e da regiao onde sera feito o deploy do kinesis - Utilizando o Google Colab faca a criacao da credencial com os nomes aws_access_key_id e aws_secret_access_key.
aws_access_key_id = userdata.get('aws_access_key_id') # getpass.getpass('Enter AWS Access Key ID: ')
aws_secret_access_key = userdata.get('aws_secret_access_key') # getpass.getpass('Enter AWS Secret Access Key: ')
aws_region = input('Enter AWS Region: ')

In [7]:
!aws configure set aws_access_key_id $aws_access_key_id
!aws configure set aws_secret_access_key $aws_secret_access_key
!aws configure set default.region $aws_region

In [8]:
!aws kinesis create-stream --stream-name eda-kds-market

In [ ]:
!aws kinesis describe-stream-summary --stream-name eda-kds-market

In [10]:
import boto3
import random
import time
import json
import datetime

In [11]:
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = aws_region
)

In [12]:
# Funcao para gerar dados aleatorios de acoes
def gerar_dados_acoes():
    simbolos = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'TSLA']
    simbolo = random.choice(simbolos)
    preco = round(random.uniform(100, 500), 2)
    volume = random.randint(1000, 10000)
    timestamp = datetime.datetime.utcnow().isoformat()
    dados = {
        'simbolo': simbolo,
        'preco': preco,
        'volume': volume,
        'timestamp': timestamp
    }
    return dados

In [13]:
# Loop para enviar dados ao Kinesis Data Stream
def enviar_dados_kinesis(stream_name, payload, partition_key):
      try:
        response = kinesis_client.put_record(
            StreamName = stream_name,
            Data = payload,
            PartitionKey = partition_key
        )

        print(f"Dados enviados: {dados}")
      except Exception as e:
        print(f"Erro ao enviar dados: {e}")
        time.sleep(5)  # Ajuste o intervalo de tempo conforme necessario

In [ ]:
if __name__ == "__main__":
  stream_name = 'eda-kds-market'

  while True:
    dados = gerar_dados_acoes()
    payload = json.dumps(dados).encode('utf-8')
    partition_key = dados['simbolo']

    enviar_dados_kinesis(stream_name, payload, partition_key)
    time.sleep(2) # Simula o envio periodico de transacoes